# CIF control NODDI analysis

#### v1.2 ~ 30/08/2019  
Niall Bourke (n.bourke@imperial.ac.uk)  
-Based on Maria Yanez Lopez original NODDI notebook  

##### Pre-requisites:
- Python 2.7 required for AMICO (Notebook in Py3)
- spams needs to be setup via conda
(https://github.com/daducci/AMICO/issues/61)
- DATA INPUT from BIDS source dir

##### Scripts needed for it to work:  
(Hardcoded and need to be ammended - could call in Jupyter cell?)
- AMICO_modeling.py    
- Normalise_bvec.py    
- hpcrunarrayjob.sh/ hpcSubmit  

#### There are issues submitting as a job on the cluster.. Modeling has been run locally

## Preparation: 

In [ ]:
import os

project = "BIO-AX-TBI"

directory = ("/rds/general/user/nbourke/ephemeral/" + project + "/data/")

if not os.path.exists(directory):
    os.makedirs(directory)
    
raw = ("/rds/general/project/c3nl_djs_imaging_data/live/data/raw/" + project + "/")
source = ("/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/")
wd = ("/rds/general/user/nbourke/ephemeral/" + project + "/")
setup = (wd + "/setup.sh" )

In [ ]:
%%writefile $setup

$(: Project label) 
project="BIO-AX-TBI" 

$(:  Where is the RAW directory?)
# export rawDir=/rds/general/project/c3nl_shared/live/${project}/sourcedata

$(: dependencies)
export dep=/rds/general/project/c3nl_shared/live/dependencies/
export templates=/rds/general/user/nbourke/home/templates
export wd=/rds/general/user/nbourke/ephemeral/${project}

# Define modules    
fsl="module load fsl";

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup
echo $wd

Data valadation - check data is correctly pulled from raw to source data

In [ ]:
import os
from os import path
import fnmatch
    
for subject in os.listdir(raw):
    #print(subject)  
    subDir = (source + "sub-" + subject + "/")
    
    try:
        for session in os.listdir(subDir):
            in_file = (subDir + session + "/dwi/msdwi/sub-" + subject + "_" + session + "_msdwi.nii")

            try:
                f = open(in_file)
                print(in_file + ' File exists')
                f.close()
            except FileNotFoundError:
                print(in_file + ' WARNING: File does not exist')
    except FileNotFoundError:
        print(subject + ' WARNING: SUBJECT NOT FOUND')
    except NotADirectoryError:
        print(subject + ' WARNING: Directory NOT FOUND')
                    

### Stage 1: File management - create an organised data directory
#### Pulling from BIDS sourcedata structure 

In [ ]:
print(project)

In [ ]:
%%script bash

source=/rds/general/project/c3nl_djs_imaging_data/live/data/sourcedata/;
project=BIO-AX-TBI ; # EDIT HERE
modeled=/rds/general/project/c3nl_djs_imaging_data/live/analysis/NODDI/BIOAX_NODDI/AMICO/modeled_data


wd=${EPHEMERAL}/${project}/;
echo -n "" > ${wd}/msdwiMissingData.txt;

for subject in `ls /rds/general/project/c3nl_djs_imaging_data/live/data/raw/${project}`; 
do 
    if [ ! -d ${source}/sub-${subject} ]; then 
        echo "${subject} not found in sourcedata"
    else
        
        for session in `ls ${source}/sub-${subject}/`;
        do
            #if [ ! -d $modeled/${session} ]; then
            if [ ! -d $wd/${session} ]; then

                if [ ! -d ${source}/sub-${subject}/${session}/dwi/msdwi ]; then 
                    echo "msdwi data not found for ${session}" >> ${wd}/msdwiMissingData.txt;
                else
                #ls ${source}/sub-${subject}/${session}/dwi/msdwi/*_v3_msdwi.nii
                    mkdir -p ${wd}/data/sub-${subject}/${session}/dwi/msdwi
                    mkdir -p ${wd}/data/sub-${subject}/${session}/anat/T1w
                    cp ${source}/sub-${subject}/${session}/dwi/msdwi/*  ${wd}/data/sub-${subject}/${session}/dwi/msdwi/;
                    cp ${source}/sub-${subject}/${session}/anat/T1w/*  ${wd}/data/sub-${subject}/${session}/anat/T1w/;
                fi
            fi
        done
    fi
    
done

#### Checking the data

These have a different sequence names for some reason - add to config file and rerun
sub-CH0010001
sub-CH0010002

sub-CH0010006 has two visit 1s - first one missing bvals & bvecs so removed this session

Pilots - have B0 scan but not full protocal
CIF1655
CIF1656 - rescaned has data on second visit
CIF2188

CIF2369 - had old conversion which is now deleated, current one is fine

#### Notes on calculating acquisition parameters
*requires scan card (header unlikey to contain the right info)

### Writing acqparam.txt for each subject

In [ ]:
import os
import fnmatch


DATADIR = "/rds/general/user/nbourke/ephemeral/BIO-AX-TBI/data/"
    
for subject in os.listdir(DATADIR):
    if fnmatch.fnmatch(subject, 'sub-CIF*'):
        print(subject)
        for session in os.listdir(DATADIR+subject):
            message = '0 -1 0 0.09144\n0 1 0 0.09144\n'
            file_name =  (DATADIR+"/"+subject+"/"+session+"/dwi/msdwi/acqparams.txt")
            with open(file_name, 'w') as fp:
                fp.write(message)
    
    elif fnmatch.fnmatch(subject, 'sub-CH*'):
        print(subject)
        for session in os.listdir(DATADIR+subject):
            message = '0 -1 0 0.09398\n0 1 0 0.09398\n'
            file_name =  (DATADIR+"/"+subject+"/"+session+"/dwi/msdwi/acqparams.txt")
            with open(file_name, 'w') as fp:
                fp.write(message)
        

#### Set index file 

- reads volumes and sets text file (not a single hardcoded file as some scanns had 99vol and others 100)
- index file contains 1 for num of volumes 

In [ ]:
import numpy as np
import nibabel as nb
import os
import os.path
from os import path
    
DATADIR = "/rds/general/user/nbourke/ephemeral/BIO-AX-TBI/data/"
    
for subject in os.listdir(DATADIR):
    for session in os.listdir(DATADIR+subject):
        out_file = os.path.abspath(DATADIR+subject+"/"+session+'/dwi/msdwi/index.txt')
        in_file=(DATADIR+subject+"/"+session+"/dwi/msdwi/"+subject+"_"+session+"_msdwi.nii")
        
        try:
            print (in_file)
            vols = nb.load(in_file).get_data().shape[-1]
            print(vols)
            np.savetxt(out_file, np.ones((vols,), dtype=np.int), fmt="%i", newline=" ")
        except IOError:
            print("WARNING " + in_file + " NOT FOUND!!")
            
        
#         if in_file is None:
#             print (in_file, "File does not exist") 
#         else:
#             print (in_file)
#             vols = nb.load(in_file).get_data().shape[-1]
#             print(vols)
#             np.savetxt(out_file, np.ones((vols,), dtype=np.int), fmt="%i", newline=" ")

### Stage 2: Pre-processing diffusion data

### Correct for eddy distortion and motion in the data
#### Topup (requires B0 reverse)

In [ ]:
%%script bash

wd=${EPHEMERAL}/BIO-AX-TBI;
DATADIR=${wd}/data;
echo -n "" > ${wd}/topup.txt;

sub=`ls $DATADIR`;
for subject in `echo $sub`; 
do  
    
    for session in `ls ${DATADIR}/${subject}/`;
    do  
        filePath=${DATADIR}/${subject}/${session}/dwi/msdwi; 
        echo "module load fsl; fslroi ${filePath}/${subject}_${session}_msdwi.nii ${filePath}/${subject}_${session}_AP_B0.nii 0 1; fslmerge -t ${filePath}/${subject}_${session}_combined_AP_PA_B0 ${filePath}/${subject}_${session}_AP_B0.nii ${filePath}/${subject}_${session}_b0_reversed.nii; topup --imain=${filePath}/${subject}_${session}_combined_AP_PA_B0 --datain=${filePath}/acqparams.txt --config=${FSLDIR}/etc/flirtsch/b02b0.cnf --out=${filePath}/topup --iout=${filePath}/hifi_b0 ; fslmaths ${filePath}/hifi_b0 -Tmean ${filePath}/hifi_b0; bet ${filePath}/hifi_b0 ${filePath}/hifi_b0_brain -m" >>  ${wd}/topup.txt;   
    done;
done;

        # repol - dealing with slice drop out from motion
        # cnr = contrast noise ratio maps for QC
        
    # Run jobs
    /rds/general/project/c3nl_shared/live/dependencies/hpcSubmit ${wd}/topup.txt 03:00:00 6 8Gb; 
            

### Eddy

#### Neeed to swap brain mask here for something better like the one from FreeSurfer

In [ ]:
%%script bash

wd=${EPHEMERAL}/BIO-AX-TBI;
DATADIR=${wd}/data;
echo -n "" > ${wd}/eddy.txt;

# ----------------------------

sub=`ls $DATADIR`;
for subject in `echo $sub`; 
do  
    
    for session in `ls ${DATADIR}/${subject}/`;
    do  
        filePath=${DATADIR}/${subject}/${session}/dwi/msdwi
        # Generate a brain mask
        echo "module load fsl; eddy_openmp --imain=${filePath}/${subject}_${session}_msdwi.nii --mask=${filePath}/hifi_b0_brain_mask --acqp=${filePath}/acqparams.txt --index=${filePath}/index.txt --bvecs=${filePath}/${subject}_${session}_msdwi.bvec --bvals=${filePath}/${subject}_${session}_msdwi.bval --topup=${filePath}/topup --repol --cnr_maps --out=${filePath}/ec_data" >>  ${wd}/eddy.txt;   
    done;
done;

        # repol - dealing with slice drop out from motion
        # cnr = contrast noise ratio maps for QC
        
    # Run jobs
    /rds/general/project/c3nl_shared/live/dependencies/hpcSubmit ${wd}/eddy.txt 12:00:00 4 6Gb; 
            

### Normalise bvec  - *Not run on human data, not required!*

In [ ]:
# %%script bash

# python /rds/general/user/nbourke/home/projects/NODDI_HPC_fitting/scripts/Normalise_bvec.py

In [ ]:
# #Import python libraries
# import sys, os 
# import numpy as np
# from dipy.io import read_bvals_bvecs
# from dipy.core.geometry  import normalized_vector

# DATADIR = "/rds/general/user/nbourke/ephemeral/BIOAX_NODDI/data/"

# for subject in os.listdir(DATADIR):
#     for session in os.listdir(DATADIR+subject):
#         #Normalizing bvec file:
#         bvals, bvecs = read_bvals_bvecs(DATADIR+subject+"/"+session+"/dwi/msdwi/"+subject+"_"+session+"_msdwi.bval", DATADIR+subject+"/"+session+"/dwi/msdwi/rotated_bvec") 
#         print(DATADIR+subject+"/"+session+"/dwi/msdwi/"+subject+"_"+session+"_msdwi.bval")
#         print(DATADIR+subject+"/"+session+"/dwi/msdwi/rotated_bvec")


#         nvec = normalized_vector(bvecs)
#         #Changing the b0 bvecs to [0 0 1]:
#         nvec[np.isnan(nvec)] = 0
#         nvec[0:4,2] = 1
#         #Transposing and saving the new normalized bvec file
#         nvec = np.transpose(nvec)
#         np.savetxt(DATADIR + "/" + subject + "/" + session + "/dwi/msdwi/norm_bvec.bvec", nvec, fmt='%.18e', delimiter=' ', newline='\n')
#         continue
    
#     else:
#         continue


### AMICO is fussy on the structure of data so lets learn how to do some data manipulation in python
pathlib - py3

In [ ]:
#Import python libraries
import sys, os 
import pathlib
from shutil import copyfile

# pathlib - py3

DATADIR = "/rds/general/user/nbourke/ephemeral/BIO-AX-TBI/data/"

for subject in os.listdir(DATADIR):
    for session in os.listdir(DATADIR+subject):
        
        directory=("/rds/general/user/nbourke/ephemeral/BIO-AX-TBI/AMICO/" + session + "/")
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        #Normalizing bvec file:
        src_bval=(DATADIR+subject+"/"+session+"/dwi/msdwi/"+subject+"_"+session+"_msdwi.bval")
        src_bvec=(DATADIR+subject+"/"+session+"/dwi/msdwi/ec_data.eddy_rotated_bvecs")
        src_ec=(DATADIR+subject+"/"+session+"/dwi/msdwi/ec_data.nii.gz")
        src_mask=(DATADIR+subject+"/"+session+"/dwi/msdwi/hifi_b0_brain_mask.nii.gz")

        dst_bval=(directory + "msdwi.bval")
        dst_bvec=(directory + "rotated_bvec")
        dst_ec=(directory + "ec_data.nii.gz")
        dst_mask=(directory + "hifi_b0_brain_mask.nii.gz")
        
        #print(src_bval)
        #print(dst_bval)
        
        
        bvalPath = pathlib.Path(src_bval)
        if bvalPath.exists ():
            copyfile(src_bval, dst_bval)
        else:
            print (src_bval, "File not exist")
        
        bvecPath = pathlib.Path(src_bvec)
        if bvecPath.exists ():
            copyfile(src_bvec, dst_bvec)
        else:
            print (src_bvec, "File not exist")
        
        ecPath = pathlib.Path(src_ec)
        if ecPath.exists ():
            copyfile(src_ec, dst_ec)
        else:
            print (src_ec, "File not exist")
        
        maskPath = pathlib.Path(src_mask)
        if maskPath.exists ():
            copyfile(src_mask, dst_mask)
        else:
            print (src_mask, "File not exist")
    
    else:
        continue
    


### Run AMICO python NODDI fitting, submitting to the cluster

This step uses https://github.com/daducci/AMICO/blob/master/doc/demos/NODDI_01.md.

* Beware there is some hard coding in the python script that is called!  
* This didnt work on cluster so ran it locally 

In [ ]:
%%bash

#import amico 

echo -n "" > ~/NODDI_fitting.txt

dataDir=/rds/general/user/nbourke/home/CIF-CONTROLS/AMICO/

for subject in `ls $dataDir`; do 
    echo "python ${HOME}/projects/NODDI_HPC_fitting/scripts/AMICO_modeling.py $dataDir $subject" >> ~/NODDI_fitting.txt
done
    
    # Run jobs
    /rds/general/project/c3nl_shared/live/dependencies/hpcSubmit ~/NODDI_fitting.txt 01:00:00 1 2Gb
            

### Check the state of the submitted cluster jobs

In [ ]:
!qstat

### Clean output and error files after checking them

----------

# Stage 2 - Moving the images to standard space, to be able to do statistics...

## This is a quick registration using fsl, may want to go an extra step and use DTI-TK
** HACKED, Not standard format

In [ ]:
%%script bash

dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=~/March2020/data;
echo -n "" > ${wd}/dtifitJob.txt
job=${wd}/dtifitJob.txt

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
# ----------------------------

for visit in v1 v2 v3 hc rugby unknown ; 
    do 
    
    for subject in `ls ${wd}/${visit}/`; do
        #echo $subject
        echo "module load fsl; dtifit --data=${wd}/${visit}/${subject}/ec_data.nii.gz --out=${wd}/${visit}/${subject}/dti --mask=${wd}/${visit}/${subject}/hifi_b0_brain_mask.nii --bvecs=${wd}/${visit}/${subject}/rotated_bvec --bvals=${wd}/${visit}/${subject}/msdwi.bval -w" >> ${job}   
    done;
done;

    # Run job
    ${dep}/hpcSubmit ${job} 02:00:00 1 2Gb
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}    

## Getting CORRECT fa scalers & Register

In [ ]:
%%script bash

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=~/March2020/data;

export DTITK_ROOT=/apps/dti-tk/2.3.1/; export PATH=$PATH:$DTITK_ROOT/bin:$DTITK_ROOT/utilities:$DTITK_ROOT/scripts:$DTITK_ROOT/lib:$DTITK_ROOT/include;

echo -n "" > echo -n "" > ${wd}/flirt.txt
job=${wd}/flirt.txt
# ----------------------------

# set DTITK path #
export DTITK_ROOT=/apps/dti-tk/2.3.1/
export PATH=$PATH:$DTITK_ROOT/bin:$DTITK_ROOT/utilities:$DTITK_ROOT/scripts:$DTITK_ROOT/lib:$DTITK_ROOT/include
                    
for visit in v1 v2 v3 hc rugby unknown ; do
    
    for subject in `ls ${wd}/${visit}/`; do 
          echo "module load fsl; fsl_to_dtitk ${wd}/${visit}/${subject}/dti; TVtool -in ${wd}/${visit}/${subject}/dti_dtitk.nii.gz -out ${wd}/${visit}/${subject}/fa.nii.gz -fa ; flirt -in ${wd}/${visit}/${subject}/fa.nii.gz -ref $template -omat ${wd}/${visit}/${subject}/flirt.mat -out ${wd}/${visit}/${subject}/MNI_${subject}_fa.nii.gz" >> ${job}  
    done
done

    # Run job
    ${dep}/hpcSubmit ${job} 02:00:00 1 2Gb 
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}   
    

## Taking the mat file from the registration of the eddy corrected data to MNI space

In [ ]:
%%script bash

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=~/March2020/data;
echo -n "" > ${wd}/flirtNODDI.txt
job=${wd}/flirtNODDI.txt
# ----------------------------

for visit in v1 v2 v3 hc rugby unknown ; 
    do
    for subject in `ls ${wd}/${visit}/`; do
        for modality in `ls ${wd}/${visit}/${subject}/AMICO/NODDI/*.nii.gz`;
        do
        mod=`basename $modality`
        #echo $mod
    
        echo "module load fsl; flirt -in ${wd}/${visit}/${subject}/AMICO/NODDI/${mod} -ref $template -init ${wd}/${visit}/${subject}/flirt.mat -applyxfm -out ${wd}/${visit}/${subject}/AMICO/NODDI/MNI_${mod}.nii.gz" >> ${wd}/flirtNODDI.txt    
    
        done
    done
done
    
    # Run job
    ${dep}/hpcSubmit ${job} 02:00:00 1 2Gb 
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}   
      

## Think about the voxelwise statistical tests to be run and how the data needs to be formed for it

1. Repeated measures ANOVA in patients with three timepoints 
2. Reapeated measures ANOVE in patients with two timepoints
3. Between group comparison of patients v1,v2,v3 with controls (Three seperate analysis?)

    - Scanner as a factor..b

## CIF data - predicting LARS & FrSBe

In [ ]:
%%script bash

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=~/March2020/data/analysis;


module load fsl
# ----------------------------

for ii in `ls ${wd}`; 
    do
        ls ${wd}/${ii}/MNI_${ii}_fa.nii.gz >> ${wd}/FA_path.txt
        ls ${wd}/${ii}/AMICO/NODDI/MNI_FIT_ICVF.nii.gz >> ${wd}/ICVF_path.txt
        ls ${wd}/${ii}/AMICO/NODDI/MNI_FIT_OD.nii.gz >> ${wd}/OD_path.txt
        ls ${wd}/${ii}/AMICO/NODDI/MNI_FIT_ISOVF.nii.gz >> ${wd}/ISOVF_path.txt
done


for mod in FA ICVF OD ISOVF; 
    do
    fslmerge -t ${wd}/tbss/${mod} `cat ${wd}/${mod}_path.txt` 
done

In [ ]:
# %%script bash

# template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
# dep="/rds/general/project/c3nl_shared/live/dependencies/"
# wd=${EPHEMERAL}/Jan2020
# echo -n "" > ${wd}/flirtNODDI.txt
# job=${wd}/flirtNODDI.txt

# module load fsl
# # ----------------------------

# mkdir ${wd}/tbss

# for visit in v1 v2 v3 con ; do
    
#     for subject in `ls ${wd}/${visit}/`; do
         
#         ls  ${wd}/${visit}/${subject}/MNI_*fa.nii.gz >>  ${wd}/subj_fa.txt
        
#         for modality in `ls ${wd}/${visit}/${subject}/AMICO/NODDI/*.nii.gz`;
#         do
#         mod=`basename $modality`
#         mo=$(echo ${mod} | cut -d '.' -f1)
#         m=$(echo ${mo##*_})
#         #echo $m

#         echo "${wd}/${visit}/${subject}/AMICO/NODDI/MNI_FIT_${m}.nii.gz" >>  ${wd}/subj_${m}.txt                       
         
#         done
#     done
# done

                   
#          fslmerge -t ${wd}/tbss/all_ICVF `cat ${wd}/subj_ICVF.txt`   
#              fslmerge -t ${wd}/tbss/all_OD `cat ${wd}/subj_OD.txt`    
#                  fslmerge -t ${wd}/tbss/all_ISOVF `cat ${wd}/subj_ISOVF.txt`  
#                     fslmerge -t ${wd}/tbss/all_FA `cat ${wd}/subj_fa.txt`  
                        

###  4D files with patients accross three timepoints
#### Check what is common across all visits

In [ ]:
%%script bash

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=${EPHEMERAL}/Jan2020
echo -n "" > ${wd}/flirtNODDI.txt
job=${wd}/flirtNODDI.txt

module load fsl
# ----------------------------

# # Check what is common across all visits
# for visit in v3 ; do
#      for subject in `ls ${wd}/${visit}/`; 
#         do
#         s=${subject::(-3)}
#         sub=${s:15}
#         echo $sub >> ${wd}/longSubj.txt

#         ls  ${wd}/v1/*-${sub}_v1/MNI_*fa.nii.gz
#         ls  ${wd}/v2/*-${sub}_v2/MNI_*fa.nii.gz

#     done
# done  

echo "" > ${wd}/conSubj.txt
# Controls
for visit in HC ; do
     for subject in `ls ${wd}/${visit}/`; 
        do
        echo $subject >> ${wd}/conSubj.txt
    done
done 

### With V3

#### Empty
- CH0010024_v2  


#### unprocessed
- ses-2018-09-20-CH0010019_v1 : inconsistant b0 & b0_reverse dimensions..

- CH0010016_v1  : failing NODDI modeling

### With V2  

### Remove the missing subjects and add full paths to data in a text file

In [ ]:
%%script bash

dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=${EPHEMERAL}/Jan2020
# ----------------------------

# # Ridiculous way to loop - but it's 7am and I want it in this order to match fsl glm example  
#     for ii in `cat ${wd}/longSubj.txt`; 
#         do
#         for v in v1;
#             do
#             ls ${wd}/${v}/*-${ii}_${v}/MNI_*-${ii}_${v}_fa.nii.gz >> ${wd}/longFA_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ICVF.nii.gz >> ${wd}/longICVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_OD.nii.gz >> ${wd}/longOD_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ISOVF.nii.gz >> ${wd}/longISOVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_dir.nii.gz >> ${wd}/longDIR_path.txt
#         done
#     done
#     for ii in `cat ${wd}/longSubj.txt`; 
#         do
#         for v in v2;
#             do
#             ls ${wd}/${v}/*-${ii}_${v}/MNI_*-${ii}_${v}_fa.nii.gz >> ${wd}/longFA_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ICVF.nii.gz >> ${wd}/longICVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_OD.nii.gz >> ${wd}/longOD_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ISOVF.nii.gz >> ${wd}/longISOVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_dir.nii.gz >> ${wd}/longDIR_path.txt
#         done
#     done
#     for ii in `cat ${wd}/longSubj.txt`; 
#         do
#         for v in v3;
#             do
#             ls ${wd}/${v}/*-${ii}_${v}/MNI_*-${ii}_${v}_fa.nii.gz >> ${wd}/longFA_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ICVF.nii.gz >> ${wd}/longICVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_OD.nii.gz >> ${wd}/longOD_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_ISOVF.nii.gz >> ${wd}/longISOVF_path.txt
#             ls ${wd}/${v}/*-${ii}_${v}/AMICO/NODDI/MNI_FIT_dir.nii.gz >> ${wd}/longDIR_path.txt
#         done
#     done


    for ii in `cat ${wd}/conSubj.txt`; 
        do
        for v in HC;
            do
            ls ${wd}/${v}/${ii}/MNI_${ii}_fa.nii.gz >> ${wd}/conFA_path.txt
            ls ${wd}/${v}/${ii}/AMICO/NODDI/MNI_FIT_ICVF.nii.gz >> ${wd}/conICVF_path.txt
            ls ${wd}/${v}/${ii}/AMICO/NODDI/MNI_FIT_OD.nii.gz >> ${wd}/conOD_path.txt
            ls ${wd}/${v}/${ii}/AMICO/NODDI/MNI_FIT_ISOVF.nii.gz >> ${wd}/conISOVF_path.txt
        done
    done

### Stack volumes into 4D file to analyse

In [ ]:
%%script bash

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
dep="/rds/general/project/c3nl_shared/live/dependencies/"
wd=${EPHEMERAL}/Jan2020
echo -n "" > ${wd}/fslMerge.txt
job=${wd}/fslMerge.txt

# ----------------------------

for mod in FA ICVF OD ISOVF; 
    do
    echo "module load fsl; fslmerge -t ${wd}/tbss/con_${mod} \`cat ${wd}/tbss/con${mod}_path.txt\` " >> ${job} # \ to pretect special characters
done

    # Run job
    ${dep}/hpcSubmit ${job} 01:00:00 1 6Gb 
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}   
    

### setup design matrix and contrast as per

https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/GLM

Statistics Jargon Decoder: Repeated Measures ANOVA (2). This is again a "Repeated Measures ANOVA" with one fixed and one random factor, the same as the previous example except that the fixed factor has three levels. Fitting such a mixed effects model with Ordinary Least Squares (OLS) (as done in Feat) requires an assumption of compound symmetry. This is the state of equal variance and intra-subject correlations being equal. That is, Cov(scan1,scan2) = Cov(scan1,scan3) = Cov(scan2,scan3).

The assumption of this design, compound symmetry, is probably a reasonable assumption unless, say, the data are from a long or irregularly sampled longitudinal study. For example, if scan 1 and 2 are collected 1 week apart and scan 3 is collected 1 year later, it's unlikely that they are equally correlated. Note, also, only intrasubject contrasts are valid with this design. For example, a [1 1 1] contrast cannot be assessed; if such a contrast is of interest, the individual measures should be averaged with fslmaths and then studied in a one-sample t-test (assuming they are difference measures, and can be so suitably analysed).

### Extraction from  tbbs_3 & 4

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
wd=${EPHEMERAL}/Jan2020
${fsl}
echo -n "" > ${wd}/fslSkel.txt
job=${wd}/fslSkel.txt

# ----------------------------
    
    subProj=controls
    
    # create mean FA
    echo "creating valid mask and mean FA"
    fslmaths ${wd}/tbss/${subProj}/con_FA -max 0 -Tmin -bin ${wd}/tbss/${subProj}/mean_FA_mask -odt char
    fslmaths ${wd}/tbss/${subProj}/con_FA -mas ${wd}/tbss/mean_FA_mask${wd}/tbss/${subProj}/long_FA
    fslmaths ${wd}/tbss/${subProj}/con_FA -Tmean ${wd}/tbss/${subProj}/mean_FA

    # create skeleton
    echo "skeletonising mean FA"
    tbss_skeleton -i ${wd}/tbss/${subProj}/mean_FA -o ${wd}/tbss/${subProj}/mean_FA_skeleton

    echo "creating skeleton mask using threshold 0.2"
    fslmaths ${wd}/tbss/${subProj}/mean_FA_skeleton -thr 0.3 -bin ${wd}/tbss/${subProj}/mean_FA_skeleton_mask

    echo "creating skeleton distancemap (for use in projection search)"
    fslmaths ${wd}/tbss/${subProj}/mean_FA_mask -mul -1 -add 1 -add ${wd}/tbss/${subProj}/mean_FA_skeleton_mask ${wd}/tbss/${subProj}/mean_FA_skeleton_mask_dst
    distancemap -i ${wd}/tbss/${subProj}/mean_FA_skeleton_mask_dst -o ${wd}/tbss/${subProj}/mean_FA_skeleton_mask_dst

    echo "projecting all FA data onto skeleton"
    tbss_skeleton -i ${wd}/tbss/${subProj}/mean_FA -p 0.3 ${wd}/tbss/${subProj}/mean_FA_skeleton_mask_dst ${FSLDIR}/data/standard/LowerCingulum_1mm ${wd}/tbss/${subProj}/con_FA ${wd}/tbss/${subProj}/con_FA_skeletonised


    for ALTIM in ICVF OD ISOVF; 
        do
        echo "tbss_skeleton -i ${wd}/tbss/${subProj}/mean_FA -p 0.3 ${wd}/tbss/${subProj}/mean_FA_skeleton_mask_dst ${FSLDIR}/data/standard/LowerCingulum_1mm ${wd}/tbss/${subProj}/con_FA ${wd}/tbss/${subProj}/con_${ALTIM}_skeletonised -a ${wd}/tbss/${subProj}/con_${ALTIM}" >> ${job}   
    done
    
    
    # Run job
    ${dep}/hpcSubmit ${job} 01:00:00 1 6Gb 
    echo ""; echo "***"; echo ""; echo "Submitted commands:"
    head ${job}   
    

# Tract stats

This takes a while, should be submitted!

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
tractDir=/rds/general/user/nbourke/home/templates/Corrected_Tracts
wd=${EPHEMERAL}/Jan2020/tbss
${fsl}
# ----------------------------


# Find the tract masks
    cd ${tractDir}
    TRACTS=`ls *.gz`

# Nested for loop
for mod in ICVF OD ISOVF FA; do    
    for project in controls #longitudinal; 
        do
        
        # 1 Copy subjects
        cp ${wd}/long${mod}_path.txt ${wd}/${project}/tractStats/${mod}/AAA_subj.txt;
        
        
        # 2 Whole brain summary values
        mkdir -p ${wd}/${project}/tractStats/${mod}
        echo "wholeBrain_${mod}" > ${wd}/${project}/tractStats/${mod}/wholeBrainStats.txt
        fslstats -t ${wd}/${project}/con_${mod}_skeletonised.nii.gz -M >> ${wd}/${project}/tractStats/${mod}/wholeBrainStats.txt
        

        # 3 Tract values
        for i in $TRACTS; do
            #echo $i
            k=$(echo ${i} | cut -d '.' -f1)            
            echo ${mod}_${k} > ${wd}/${project}/tractStats/${mod}/${mod}_${k}.txt
            fslstats -t ${wd}/${project}/con_${mod}_skeletonised.nii.gz -k ${tractDir}/$i -M >> ${wd}/${project}/tractStats/${mod}/${mod}_${k}.txt   
        done
    done
    # paste  -d "," `ls $DATADIR/${project}/tractStats/${mod}/` >> $DATADIR/${project}/${mod}_tract_stats.csv    
done


### Voxelwise analysis

In [ ]:
%%bash -s "$setup"
export setup=$1;
source $setup

template=/rds/general/user/nbourke/home/templates/FMRIB58_FA_1mm.nii.gz
tractDir=/rds/general/user/nbourke/home/templates/Corrected_Tracts
wd=${EPHEMERAL}/Jan2020/tbss
${fsl}

echo -n "" > ${wd}/vx.txt
job=${wd}/vx.txt

# ----------------------------

# Nested for loop
for mod in ICVF OD ISOVF FA; do    
    for project in longitudinal; 
        do
  
        #- set output -#
        output=${wd}/${project}/randomise_output/${mod}
        mkdir -p ${output}

        #- set variables -#
        input=${wd}/${project}/con_${mod}.nii.gz
        FA_skeleton_mask=${wd}/${project}/mean_FA_skeleton_mask.nii.gz
        design=${wd}/${project}/designGLM.mat    #design/design.mat
        contrast=${wd}/${project}/designGLM.con   #design/contrast.con
        
        ~/c3nl_tools/pbs_randomise_par -wt 24:00:00 -mem 14Gb -i ${input} -o ${output}/tbss_${mod} -m $FA_skeleton_mask -d $design -t $contrast -n 5000 --T2 -V
        
    done
done

  
---


# QC data

In [ ]:
# Set data dir and copy cell into terminal

### START COPY ###

template=/Users/niallbourke/hpc/templates/FMRIB58_FA_1mm.nii.gz
DATADIR=/Users/niallbourke/hpc/eph/NODDIDATA/analysis/

for project in ukXsectional ukV1V2; do
    
    for subject in `ls $DATADIR/${project}/data/`; do
        
        fa=`ls ${DATADIR}/${project}/data/${subject}/MNI_*fa.nii.gz` 
        echo "$fa" >>  ${DATADIR}/${project}/subj_fa.txt
        
        for modality in `ls ${DATADIR}/${project}/data/${subject}/AMICO/NODDI/MNI*.nii.gz`;
        do
        
        echo "Next scan set: $subject"
        fsleyes $template $modality ;

        done;
    done;
done

### END COPY ###


# temp processing for missed subj

In [ ]:
%%script bash

wd=/rds/general/user/nbourke/home/eph/BIO-AX-TBI/missed/unproc;
DATADIR=${wd};
echo -n "" > ${wd}/topup.txt;

module load fsl

sub=sub-CH0010019
for subject in `echo $sub`;
    do
    for session in `ls ${DATADIR}/${subject}/`;
        do
        filePath=${DATADIR}/${subject}/${session}/dwi/msdwi;
        module load fsl;
        fslroi ${filePath}/${subject}_${session}_msdwi.nii ${filePath}/${subject}_${session}_AP_B0.nii 0 1;
        fslmerge -t ${filePath}/${subject}_${session}_combined_AP_PA_B0 ${filePath}/${subject}_${session}_AP_B0.nii ${filePath}/${subject}_${session}_b0_reversed.nii;
        topup --imain=${filePath}/${subject}_${session}_combined_AP_PA_B0 --datain=${filePath}/acqparams.txt --config=${FSLDIR}/etc/flirtsch/b02b0.cnf --out=${filePath}/topup --iout=${filePath}/hifi_b0 ;
        fslmaths ${filePath}/hifi_b0 -Tmean ${filePath}/hifi_b0;
        bet ${filePath}/hifi_b0 ${filePath}/hifi_b0_brain -m
        eddy_openmp --imain=${filePath}/${subject}_${session}_msdwi.nii --mask=${filePath}/hifi_b0_brain_mask --acqp=${filePath}/acqparams.txt --index=${filePath}/index.txt --bvecs=${filePath}/${subject}_${session}_msdwi.bvec --bvals=${filePath}/${subject}_${session}_msdwi.bval --topup=${filePath}/topup --repol --cnr_maps --out=${filePath}/ec_data

    done;
done;
